# Eigen Values and Eigen vectors

## what is the intuition behind these concepts? 

The idea stems from the fact that when linear transformations (matrix dot products are actually linear transformations of space, geometrically speaking) are applied, there are some vectors that are remain aligned to the line along which it is described. These vectors are called eigen vectors. The eigen vectors are scaled up or down based on the transformation, those scaling factors are called eigen values. We'll try to understand this step by step.

The cartesian plane (X-Y axis), I might be wrong here, has something called basis vectors described as (1,0) or $\hat{i}$ and (0, 1) or $\hat{j}$.  

The point of basis vectors is that we can define any point in the X-Y axis using the basis vectors. 
For example the vector (3, 2) could be obtained by scaling (or multiplying) $\hat{i}$ by 3 and $\hat{j}$ by 2. 

Infact, we describe the vector literally as 3$\hat{i}$ + 2$\hat{j}$.

Or its matrix counter part:
$$\begin{bmatrix} 1 & 0 \\\\ 0 & 1 \end{bmatrix}$$ $$\begin{bmatrix} 3 \\\\ 2 \end{bmatrix}$$

In [ ]:
from ast import TypeVarTuple
from math import prod
import torch

In [ ]:
def get_trace(matrix: torch.Tensor) -> float:
  x_indices = y_indices = torch.arange(0,matrix.shape[0], step = 1)

  diagonal_elements = matrix[x_indices, y_indices].unsqueeze(dim=0)
  trace_of_matrix = torch.sum(diagonal_elements, dim=1).item()
  return trace_of_matrix

In [ ]:
def multiply_elements_of_tensor(data: torch.Tensor, dim: int = 0, keepdim: bool = TypeVarTuple):
  """
  Set dim=0 multiplying across columns and dim=1 for across rows.
  """
  real_dim_is_one = False
  if data.dim() == 1:
    data = data.unsqueeze(dim=0)
    real_dim_is_one = True
  
  if dim == 0:
    prod_matrix = torch.empty((0,), dtype=torch.float)
    for i in range(0, data.shape[0]):
      product = torch.tensor([1.0], dtype=torch.float)
      for j in range(0, data.shape[1]):
        product *= data[i,j]
      prod_matrix = torch.cat((prod_matrix, product), dim=0)  
  else:
    prod_matrix = torch.empty((0,), dtype=torch.float)
    for j in range(0, data.shape[1]):
      product = torch.tensor([1.0], dtype=torch.float)
      for i in range(0, data.shape[0]):
        product *= data[i,j]
      prod_matrix = torch.cat((prod_matrix, product), dim=0)

  if keepdim:
    if dim == 0:
      prod_matrix = prod_matrix.view(-1, 1)
    else:
      prod_matrix = prod_matrix.view(1, -1)
  
  if real_dim_is_one:
    return prod_matrix[0,0]
  else:
    return prod_matrix
    




    

In [ ]:
def get_determinant(matrix: torch.Tensor) -> float:
    """
    Calculates the determinant of 2x2 matrix
    """
    indices = torch.arange(start=0, end=matrix.shape[0], step = 1)
    reversed_indices = torch.arange(start=matrix.shape[0]-1, end=-1, step = -1)
    major_diagonal = matrix[(indices, indices)]
    minor_diagonal = matrix[(indices, reversed_indices)]
    prod_major_diagonal = multiply_elements_of_tensor(data=major_diagonal, dim=0).item()
    prod_minor_diagonal = multiply_elements_of_tensor(data=minor_diagonal, dim=0).item()
    
    determinant = prod_major_diagonal - prod_minor_diagonal

    return determinant

In [ ]:

def find_roots(a, b, c):
  discriminant = b**2 - (4 * a * c)
  discriminant_tensor = torch.tensor([discriminant], dtype=torch.float)
  return sorted([(-b + torch.sqrt(discriminant_tensor)).item() / 2 * a, (-b - torch.sqrt(discriminant_tensor)).item() / 2 * a], key=lambda item: item)

In [ ]:

def calculate_eigenvalues(matrix: torch.Tensor) -> torch.Tensor:
    """
    Compute eigenvalues of a 2Ã2 matrix using PyTorch.
    Input: 2Ã2 tensor; Output: 1-D tensor with the two eigenvalues in ascending order.
    """
    # Your implementation here
    trace_of_matrix = get_trace(matrix=matrix)
    determinant = get_determinant(matrix=matrix)

    eigen_values = find_roots(1.0, -trace_of_matrix, determinant)
    return torch.as_tensor(eigen_values, dtype=torch.float)